In [160]:
import pandas as pd
import os

In [161]:
input_dir = "./input/"

df_all_current_year = pd.read_csv(
    input_dir + "current_year_obs_merged.csv"
)

df_taxonomy = pd.read_csv(
    input_dir + "ebird-taxonomy.csv"
)

df_subnational1_names = pd.read_csv(
    input_dir + "subnational1.csv"
)

df_subnational1_boundaries = pd.read_csv(
    input_dir + "subnational1_boundaries.csv"
)

df_family_descriptions = pd.read_csv(
    input_dir + "family_descriptions.csv"
)

df_bird_name_description_img = pd.read_csv(
    input_dir + "bird_name_description_image.csv"
)

print("All Columns for Current Year Observations: ")
print(df_all_current_year.columns)
print("All Columns for Taxonomy: ")
print(df_taxonomy.columns)
print("All Columns for Subnational 1: ")
print(df_subnational1_names.columns)
print("All Columns for Subnational 1 Boundaries: ")
print(df_subnational1_boundaries.columns)
print("All Columns for Family Descriptions: ")
print(df_family_descriptions.columns)
print("All Columns for Bird Description and Image: ")
print(df_bird_name_description_img.columns)

All Columns for Current Year Observations: 
Index(['speciesCode', 'comName', 'sciName', 'locId', 'locName', 'obsDt',
       'howMany', 'lat', 'lng', 'obsValid', 'obsReviewed', 'locationPrivate',
       'subId', 'subnational2Code', 'subnational2Name', 'subnational1Code',
       'subnational1Name', 'countryCode', 'countryName', 'userDisplayName',
       'obsId', 'checklistId', 'presenceNoted', 'hasComments', 'firstName',
       'lastName', 'hasRichMedia', 'exoticCategory'],
      dtype='object')
All Columns for Taxonomy: 
Index(['SCIENTIFIC_NAME', 'COMMON_NAME', 'SPECIES_CODE', 'CATEGORY',
       'TAXON_ORDER', 'COM_NAME_CODES', 'SCI_NAME_CODES', 'BANDING_CODES',
       'ORDER', 'FAMILY_COM_NAME', 'FAMILY_SCI_NAME', 'REPORT_AS', 'EXTINCT',
       'EXTINCT_YEAR', 'FAMILY_CODE'],
      dtype='object')
All Columns for Subnational 1: 
Index(['REGION_CODE', 'REGION_NAME'], dtype='object')
All Columns for Subnational 1 Boundaries: 
Index(['region_code', 'region', 'minX', 'maxX', 'minY', 'maxY'

In [162]:
# Missing family_description (web scraping)
family_without_descriptions = [
    "FAMILY_CODE",
    "FAMILY_SCI_NAME",
    "FAMILY_COM_NAME"
]

family_descriptions = [
    "family_scientific_name",
    "family_description"
]

# Missing species_description, species_img (web scraping)
species_calc = [
    "SPECIES_CODE", 
    "FAMILY_CODE", 
    "SCIENTIFIC_NAME",
    "COMMON_NAME", 
    "EXTINCT",
    "EXTINCT_YEAR"
]

species = [
    "species_code", 
    "family_code", 
    "scientific_name",
    "common_name", 
    "species_description",
    "species_img_link",
    "extinct",
    "extinct_year"
]

# using subId for user_id
ebird_user_calc = [
    "firstName",
    "lastName"
]

ebird_user = [
    "user_id",
    "first_name",
    "last_name"
]

subnational1_names = [
    "REGION_CODE",
    "REGION_NAME"
]

subnational1_boundaries = [
    "region_code",
    "minX",
    "maxX",
    "minY",
    "maxY"
]

subnational1 = [
    "REGION_CODE",
    "REGION_NAME",
    "minX",
    "maxX",
    "minY",
    "maxY"
]

subnational2 = [
    "subnational2Code",
    "subnational1Code",
    "subnational2Name"
]

ebird_location = [
    "locId",
    "subnational2Code",
    "locName",
    "lat",
    "lng",
    "locationPrivate"
]

observation_calc = [
    "obsId",
    "speciesCode",
    "firstName",
    "lastName",
    "locId",
    "obsDt",
    "howMany",
    "obsValid",
    "obsReviewed"
]

observation = [
    "observation_id",
    "species_code",
    "user_id",
    "location_id",
    "observation_date",
    "observation_count",
    "observation_valid",
    "observation_reviewed"
]

In [163]:
df_family_wo_descriptions = \
    df_taxonomy[family_without_descriptions] \
    .drop_duplicates() \
    .rename(columns={
        "FAMILY_CODE": "family_code",
        "FAMILY_SCI_NAME": "family_scientific_name",
        "FAMILY_COM_NAME": "family_common_name"
    }) 

df_family = df_family_wo_descriptions.merge(
        df_family_descriptions[family_descriptions],
        how="left",
        left_on="family_scientific_name",
        right_on="family_scientific_name",
    ) \
    .dropna(how="all")

df_species_wo_descriptions_n_img = \
    df_taxonomy[species_calc].drop_duplicates() \
    .rename(columns={
        "SPECIES_CODE": "species_code",
        "FAMILY_CODE": "family_code",
        "SCIENTIFIC_NAME": "scientific_name",
        "COMMON_NAME": "common_name",
        "EXTINCT": "extinct",
        "EXTINCT_YEAR": "extinct_year"
    }) \
    .dropna(how="all")
df_species_wo_descriptions_n_img["extinct"] = \
    df_species_wo_descriptions_n_img["extinct"] \
        .apply(lambda x: 1 if x == "TRUE" else 0)
df_species_wo_descriptions_n_img["extinct_year"] = \
    df_species_wo_descriptions_n_img["extinct_year"] \
        .apply(lambda x: pd.NA if pd.isna(x) else int(x))
df_bird_name_description_img["key"] = \
    df_bird_name_description_img["Bird Species"] \
        .str.replace(" ", "").str.lower()
df_species_wo_descriptions_n_img["key"] = \
    df_species_wo_descriptions_n_img["common_name"] \
        .str.replace(" ", "").str.lower()
df_species = df_species_wo_descriptions_n_img.merge(
    df_bird_name_description_img,
    how="left",
    left_on="key",
    right_on="key"
).rename(columns={
    "Description": "species_description",
    "Image link": "species_img_link"
})[species]

df_ebird_user = df_all_current_year[ebird_user_calc] \
    .rename(columns={
        "firstName": "first_name",
        "lastName": "last_name"
    }) \
    .dropna(how="all")
df_ebird_user["user_id"] = df_ebird_user["first_name"] + df_ebird_user["last_name"]
df_ebird_user["user_id"] = \
    df_ebird_user["user_id"].str.replace(" ", "").str.lower()
df_ebird_user = df_ebird_user[ebird_user] \
    .drop_duplicates(subset=["user_id"])

df_subnational1 = df_subnational1_names[subnational1_names] \
    .merge(
        df_subnational1_boundaries[subnational1_boundaries],
        how="left",
        left_on="REGION_CODE",
        right_on="region_code"
    ) \
    .drop_duplicates()[subnational1] \
    .rename(columns={
        "REGION_CODE": "subnational1_code",
        "REGION_NAME": "subnational1_name",
        "minX": "min_x",
        "maxX": "max_x",
        "minY": "min_y",
        "maxY": "max_y"
    }) \
    .dropna(how="all")

df_subnational2 = df_all_current_year[subnational2] \
    .drop_duplicates() \
    .rename(columns={
        "subnational2Code": "subnational2_code",
        "subnational1Code": "subnational1_code",
        "subnational2Name": "subnational2_name"
    }) \
    .dropna(subset="subnational2_code")

df_ebird_location = df_all_current_year[ebird_location] \
    .drop_duplicates() \
    .rename(columns={
        "locId": "location_id",
        "subnational2Code": "subnational2_code",
        "locName": "location_name",
        "lat": "latitude",
        "lng": "longitude",
        "locationPrivate": "location_private"
    }) \
    .dropna(how="all")
df_ebird_location["location_private"] = \
    df_ebird_location["location_private"] \
        .apply(lambda x: 1 if x == True else 0)
df_ebird_location["location_name"] = \
    df_ebird_location["location_name"] \
        .str.replace(r"\\", "", regex=True)
    
df_observation = df_all_current_year[observation_calc] \
    .rename(columns={
        "obsId": "observation_id",
        "speciesCode": "species_code",
        "firstName": "first_name",
        "lastName": "last_name",
        "locId": "location_id",
        "obsDt": "observation_date",
        "howMany": "observation_count",
        "obsValid": "observation_valid",
        "obsReviewed": "observation_reviewed"
    }) \
    .dropna(how="all")
df_observation["observation_id"] = \
    df_observation["observation_id"].str.replace("OBS", "O")
df_observation["observation_valid"] = \
    df_observation["observation_valid"].apply(lambda x: 1 if x == True else 0)
df_observation["observation_reviewed"] = \
    df_observation["observation_reviewed"].apply(lambda x: 1 if x == True else 0)
df_observation["observation_count"] = \
    df_observation["observation_count"] \
        .apply(lambda x: int(x) if not pd.isna(x) and int(x) > 1 else 1)
df_observation["user_id"] = df_observation["first_name"] + df_observation["last_name"]
df_observation["user_id"] = \
    df_observation["user_id"].str.replace(" ", "").str.lower()
df_observation = df_observation[observation] \
    .drop_duplicates()

output_dir = "./decomposed/"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

df_family.to_csv(
    output_dir + "ebird_family.csv",
    index=False,
    header=True,
    encoding="utf-8-sig"
)

df_species.to_csv(
    output_dir + \
        "ebird_species.csv",
    index=False,
    header=True
)

df_ebird_user.to_csv(
    output_dir + "ebird_user.csv",
    index=False,
    header=True
)

df_subnational1.to_csv(
    output_dir + "ebird_subnational1.csv",
    index=False,
    header=True
)

df_subnational2.to_csv(
    output_dir + "ebird_subnational2.csv",
    index=False,
    header=True
)

df_ebird_location.to_csv(
    output_dir + "ebird_location.csv",
    index=False,
    header=True
)

df_observation.to_csv(
    output_dir + "ebird_observation.csv",
    index=False,
    header=True
)